In [1]:
import io
import pandas as pd
import psycopg
import time

# 数据库配置
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB   = "taxi"
PG_USER = "postgres"
PG_PASS = "123"

def load_cleaned_parquet_to_postgres(path, max_rows=100000):
    """
    把清洗好的 yellow_taxi parquet 文件
    快速批量导入到 PostgreSQL 表 yellow_taxi_clean
    
    参数:
        path: parquet 文件路径
        max_rows: 最大导入行数，None 表示导入全部数据
    """
    print(f"Loading cleaned data from {path}")
    if max_rows:
        print(f"⚠️  测试模式: 仅导入前 {max_rows:,} 行")
    start_time = time.time()

    # 1. 读取 parquet（添加 max_rows 限制）
    df = pd.read_parquet(path, engine="pyarrow")
    if max_rows:
        df = df.head(max_rows)
    
    load_time = time.time() - start_time
    print(f"Loaded parquet in {load_time:.1f}s - Shape: {df.shape}")

    # 2. 列名统一成小写，方便对齐 SQL
    df.columns = [c.lower() for c in df.columns]

    # 3. 确保时间列是 datetime 类型
    if "tpep_pickup_datetime" in df.columns:
        df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    if "tpep_dropoff_datetime" in df.columns:
        df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

    # 4. 定义要导入的列，顺序必须和 yellow_taxi_clean 表一致
    cols = [
        "vendorid",
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "passenger_count",
        "trip_distance",
        "ratecodeid",
        "pulocationid",
        "dolocationid",
        "payment_type",
        "fare_amount",
        "extra",
        "mta_tax",
        "tip_amount",
        "tolls_amount",
        "improvement_surcharge",
        "total_amount",
        "congestion_surcharge"
    ]

    # 5. 只保留需要的列
    copy_df = df[cols].copy()
    
    # 6. 确保整数列的数据类型正确（避免 1.0 这样的浮点表示）
    int_cols = ["vendorid", "passenger_count", "ratecodeid", "pulocationid", "dolocationid", "payment_type"]
    for col in int_cols:
        if col in copy_df.columns:
            # 将有效值转为整数，保持 NaN 不变
            copy_df[col] = copy_df[col].apply(lambda x: int(x) if pd.notna(x) else None)
    
    # 7. 处理其余 NaN -> NULL
    copy_df = copy_df.where(pd.notnull(copy_df), None)

    prep_time = time.time() - start_time - load_time
    print(f"Data prep completed in {prep_time:.1f}s - Rows to insert: {len(copy_df):,}")

    # 8. 写入内存 CSV buffer（供 COPY 使用）
    buffer = io.StringIO()
    copy_df.to_csv(buffer, index=False, header=False, na_rep='\\N')
    buffer.seek(0)

    csv_time = time.time() - start_time - load_time - prep_time
    print(f"CSV buffer created in {csv_time:.1f}s")

    # 9. 用 COPY 批量插入 PostgreSQL
    insert_start = time.time()
    conn = psycopg.connect(
        host=PG_HOST,
        port=PG_PORT,
        dbname=PG_DB,
        user=PG_USER,
        password=PG_PASS
    )
    
    try:
        with conn.cursor() as cur:
            # 使用 copy() 方法的正确方式
            with cur.copy(f"COPY yellow_taxi_clean({', '.join(cols)}) FROM STDIN WITH (FORMAT CSV, NULL '\\N')") as copy:
                while True:
                    data = buffer.read(8192)  # 每次读取 8KB
                    if not data:
                        break
                    copy.write(data)
        
        # 显式提交
        conn.commit()
        
        insert_time = time.time() - insert_start
        total_time = time.time() - start_time
        rows_per_sec = len(copy_df) / insert_time if insert_time > 0 else 0
        
        print(f"✅ Inserted {len(copy_df):,} rows in {insert_time:.1f}s ({rows_per_sec:,.0f} rows/sec)")
        print(f"Total time: {total_time:.1f}s")
        
    except Exception as e:
        conn.rollback()
        print(f"❌ COPY failed: {e}")
        raise
    finally:
        conn.close()


# 主程序
if __name__ == "__main__":
    # 先创建表（如果还没创建）
    print("=== 创建表 yellow_taxi_clean ===")
    with psycopg.connect(
        host=PG_HOST, 
        port=PG_PORT, 
        dbname=PG_DB, 
        user=PG_USER, 
        password=PG_PASS, 
        autocommit=True
    ) as conn:
        with conn.cursor() as cur:
            cur.execute('''
                DROP TABLE IF EXISTS yellow_taxi_clean;

                CREATE TABLE yellow_taxi_clean (
                    VendorID INTEGER,
                    tpep_pickup_datetime TIMESTAMP,
                    tpep_dropoff_datetime TIMESTAMP,
                    passenger_count INTEGER,
                    trip_distance NUMERIC,
                    RatecodeID INTEGER,
                    PULocationID INTEGER,
                    DOLocationID INTEGER,
                    payment_type INTEGER,
                    fare_amount NUMERIC,
                    extra NUMERIC,
                    mta_tax NUMERIC,
                    tip_amount NUMERIC,
                    tolls_amount NUMERIC,
                    improvement_surcharge NUMERIC,
                    total_amount NUMERIC,
                    congestion_surcharge NUMERIC
                );
            ''')
    print('✅ Table yellow_taxi_clean created\n')
    
    # 导入数据 - 测试模式：只导入10万条
    print("=== 开始导入数据（测试模式）===")
    parquet_file = "yellow_taxi_2022_2025_final_cleaned.parquet"
    load_cleaned_parquet_to_postgres(parquet_file, max_rows=1000000)  # 👈 限制10万行
    
    # 验证数据
    print("\n=== 验证导入结果 ===")
    with psycopg.connect(
        host=PG_HOST, 
        port=PG_PORT, 
        dbname=PG_DB, 
        user=PG_USER, 
        password=PG_PASS
    ) as conn:
        with conn.cursor() as cur:
            # 总记录数
            cur.execute("SELECT COUNT(*) FROM yellow_taxi_clean;")
            count = cur.fetchone()[0]
            print(f"表中总记录数: {count:,}")
            
            # 时间范围
            cur.execute("""
                SELECT 
                    MIN(tpep_pickup_datetime) as earliest,
                    MAX(tpep_pickup_datetime) as latest
                FROM yellow_taxi_clean;
            """)
            date_range = cur.fetchone()
            print(f"时间范围: {date_range[0]} ~ {date_range[1]}")
            
            # NULL值统计
            cur.execute("""
                SELECT 
                    COUNT(*) - COUNT(passenger_count) as null_passenger,
                    COUNT(*) - COUNT(trip_distance) as null_distance,
                    COUNT(*) - COUNT(fare_amount) as null_fare
                FROM yellow_taxi_clean;
            """)
            nulls = cur.fetchone()
            print(f"NULL值统计: passenger={nulls[0]}, distance={nulls[1]}, fare={nulls[2]}")
            
            # 查看前5条
            cur.execute("SELECT * FROM yellow_taxi_clean LIMIT 5;")
            print("\n前5条记录:")
            for row in cur.fetchall():
                print(row)
    
    print("\n✅ 导入完成！")
    print("\n💡 提示: 测试完成后，如需导入全部数据，请将 max_rows=100000 改为 max_rows=None")

=== 创建表 yellow_taxi_clean ===
✅ Table yellow_taxi_clean created

=== 开始导入数据（测试模式）===
Loading cleaned data from yellow_taxi_2022_2025_final_cleaned.parquet
⚠️  测试模式: 仅导入前 1,000,000 行
Loaded parquet in 42.2s - Shape: (1000000, 17)
Data prep completed in 4.2s - Rows to insert: 1,000,000
CSV buffer created in 5.6s
✅ Inserted 1,000,000 rows in 3.8s (266,488 rows/sec)
Total time: 55.8s

=== 验证导入结果 ===
表中总记录数: 1,000,000
时间范围: 2022-10-01 00:00:00 ~ 2022-10-10 22:45:44
NULL值统计: passenger=0, distance=0, fare=0

前5条记录:
(2, datetime.datetime(2022, 10, 1, 0, 0), datetime.datetime(2022, 10, 1, 0, 18, 21), 1, Decimal('9.06'), 1, 138, 107, 1, Decimal('26.5'), Decimal('0.5'), Decimal('0.5'), Decimal('6.0'), Decimal('6.55'), Decimal('0.3'), Decimal('44.1'), Decimal('2.5'))
(2, datetime.datetime(2022, 10, 1, 0, 0), datetime.datetime(2022, 10, 1, 0, 10, 30), 1, Decimal('2.28'), 1, 164, 237, 1, Decimal('9.5'), Decimal('0.5'), Decimal('0.5'), Decimal('2.66'), Decimal('0.0'), Decimal('0.3'), Decimal('15.96')

In [3]:
import psycopg
import pandas as pd

# 数据库配置
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "taxi",
    "user": "postgres",
    "password": "123"
}

# 1. 读取 CSV 文件
print("📖 读取 taxi_zone_lookup.csv...")
df = pd.read_csv('taxi_zone_lookup.csv')

print(f"✅ 读取成功，共 {len(df)} 条记录")
print(f"📊 列名: {list(df.columns)}")
print("\n前 5 条数据:")
print(df.head())

# 2. 创建表
print("\n" + "="*60)
print("🗄️  创建 taxi_zone_lookup 表...")

with psycopg.connect(**PG_CONFIG) as conn:
    with conn.cursor() as cur:
        # 删除旧表（如果存在）
        cur.execute("DROP TABLE IF EXISTS taxi_zone_lookup;")
        
        # 创建新表
        cur.execute("""
            CREATE TABLE taxi_zone_lookup (
                LocationID INTEGER PRIMARY KEY,
                Borough VARCHAR(50),
                Zone VARCHAR(100),
                service_zone VARCHAR(50)
            );
        """)
        
        conn.commit()
        print("✅ 表创建成功")

# 3. 导入数据
print("\n" + "="*60)
print("📥 导入数据到数据库...")

# 统一列名（如果 CSV 列名不一样）
# 通常 CSV 可能是 LocationID, Borough, Zone, service_zone
# 或者 location_id, borough, zone, service_zone
df.columns = df.columns.str.strip()  # 去掉空格

# 重命名列以匹配数据库（如果需要）
column_mapping = {
    'LocationID': 'LocationID',
    'location_id': 'LocationID',
    'LOCATIONID': 'LocationID',
    'Borough': 'Borough',
    'borough': 'Borough',
    'BOROUGH': 'Borough',
    'Zone': 'Zone',
    'zone': 'Zone',
    'ZONE': 'Zone',
    'service_zone': 'service_zone',
    'Service_Zone': 'service_zone',
    'SERVICE_ZONE': 'service_zone'
}

# 应用列名映射
for old_name, new_name in column_mapping.items():
    if old_name in df.columns:
        df.rename(columns={old_name: new_name}, inplace=True)

# 确保列名正确
expected_cols = ['LocationID', 'Borough', 'Zone', 'service_zone']
df = df[expected_cols]

print(f"📝 准备插入 {len(df)} 条记录...")

# 使用 executemany 批量插入
with psycopg.connect(**PG_CONFIG) as conn:
    with conn.cursor() as cur:
        # 准备插入语句
        insert_query = """
            INSERT INTO taxi_zone_lookup (LocationID, Borough, Zone, service_zone)
            VALUES (%s, %s, %s, %s);
        """
        
        # 转换为列表
        data = df.values.tolist()
        
        # 批量插入
        cur.executemany(insert_query, data)
        conn.commit()
        
        print(f"✅ 成功插入 {len(df)} 条记录")

# 4. 验证导入
print("\n" + "="*60)
print("🔍 验证导入结果...")

with psycopg.connect(**PG_CONFIG) as conn:
    with conn.cursor() as cur:
        # 总数
        cur.execute("SELECT COUNT(*) FROM taxi_zone_lookup;")
        count = cur.fetchone()[0]
        print(f"✅ 表中总记录数: {count}")
        
        # 查看前 5 条
        cur.execute("SELECT * FROM taxi_zone_lookup ORDER BY LocationID LIMIT 5;")
        print("\n前 5 条记录:")
        for row in cur.fetchall():
            print(f"  ID: {row[0]}, Borough: {row[1]}, Zone: {row[2]}, Service: {row[3]}")
        
        # 按 Borough 统计
        cur.execute("""
            SELECT Borough, COUNT(*) as zone_count 
            FROM taxi_zone_lookup 
            GROUP BY Borough 
            ORDER BY zone_count DESC;
        """)
        print("\n按行政区统计:")
        for row in cur.fetchall():
            print(f"  {row[0]}: {row[1]} 个区域")

print("\n" + "="*60)
print("🎉 导入完成！现在可以使用费用计算器了！")
print("="*60)

📖 读取 taxi_zone_lookup.csv...
✅ 读取成功，共 265 条记录
📊 列名: ['LocationID', 'Borough', 'Zone', 'service_zone']

前 5 条数据:
   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone

🗄️  创建 taxi_zone_lookup 表...
✅ 表创建成功

📥 导入数据到数据库...
📝 准备插入 265 条记录...
✅ 成功插入 265 条记录

🔍 验证导入结果...
✅ 表中总记录数: 265

前 5 条记录:
  ID: 1, Borough: EWR, Zone: Newark Airport, Service: EWR
  ID: 2, Borough: Queens, Zone: Jamaica Bay, Service: Boro Zone
  ID: 3, Borough: Bronx, Zone: Allerton/Pelham Gardens, Service: Boro Zone
  ID: 4, Borough: Manhattan, Zone: Alphabet City, Service: Yellow Zone
  ID: 5, Borough: Staten Island, Zone: Arden Heights, Service: Boro Zone

按行政区统计:
  Queens: 69 